# Get Started with MLflow + Tensorflow

**Author:** Chen Qian<br>
**Date created:** 2023/08/25<br>
**Last modified:** 2023/08/25<br>

In this guide, we will show how to train your model with Tensorflow and log your training using MLflow.

For metrics visualization, we will use [Databricks Community Edition](https://community.cloud.databricks.com/), which is completely free. If you haven't, please register an account via [link](https://www.databricks.com/try-databricks), we will use it later.

We recommend turn on the free-tier GPU in Colab by **Edit -> notebook settings -> Hardware Accelerator**, it will significanly shorten the time cost.

## Install dependencies

Let's install the `mlflow` package.

In [ ]:
!pip install -q git+https://github.com/chenmoneygithub/mlflow.git@demo-tf-callback

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

## Load the dataset

We will do a simple image classification on handwritten digits with [mnist dataset](https://en.wikipedia.org/wiki/MNIST_database).

Let's load the dataset using `tensorflow_datasets` (`tfds`), which returns datasets in the format of `tf.data.Dataset`.

In [ ]:
# Load the mnist dataset.
train_ds, test_ds = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Let's preprocess our data with the following steps:
- Scale each pixel's value to `[0, 1)`.
- Batch the dataset.
- Use `prefetch` to speed up the training.

In [ ]:
def preprocess_fn(data):
    image = tf.cast(data["image"], tf.float32) / 255
    label = data["label"]
    return (image, label)

train_ds = train_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)

## Define the Model

Let's define a convolutional neural network as our classifier. We can use `keras.Sequential` to stack up the layers.

In [ ]:
input_shape = (28, 28, 1)
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

Set training-related configs, optimizers, loss function, metrics.

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

## Set up tracking/visualization tool

If you have not, please register an account of [Databricks community edition](https://www.databricks.com/try-databricks#account). It should take no longer than 1min to register.

Databricks CE (community edition) is a free platform for users to try out Databricks features. For this guide, we need the ML experiment dashboard for us to track our training progress.

After you have sucessfully registered an account, all you need to do is to run the command below to connect from Google Colab to your Databricks account. You will need to enter following information at prompt:
- **Databricks Host**: https://community.cloud.databricks.com/
- **Username**: your signed up email
- **Password**: your password

In [ ]:
!databricks configure

Databricks Host (should begin with https://): https://community.cloud.databricks.com/
Username: qianchen94era@gmail.com
Password: 
Repeat for confirmation: 


Now this colab is connected to the hosted tracking server. Let's configure MLflow metadata. Two things to set up:
- `mlflow.set_tracking_uri`: always use "databricks".
- `mlflow.set_experiment`: pick up a name you like, start with `/`.

In [ ]:
import mlflow

# This is always "databricks" when using a databricks hosted tracking server.
mlflow.set_tracking_uri("databricks")
# Pick up a name you like.
mlflow.set_experiment("/mlflow-tf-keras-mnist")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3750806661516370', creation_time=1692828174839, experiment_id='3750806661516370', last_update_time=1692919450886, lifecycle_stage='active', name='/mlflow-tf-keras-mnist', tags={'mlflow.experiment.sourceName': '/mlflow-tf-keras-mnist',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'qianchen94era@gmail.com',
 'mlflow.ownerId': '3209978630771139'}>

## Log with MLflow

There are two ways you can log to MLflow from your Tensorflow pipeline:
- MLflow auto logging.
- Use a callback.

Auto logging is simple to configure, but gives you less control. Using a callback is more flexible. Let's see how each way is done.

### MLflow Auto Logging

All you need to do is to call `mlflow.tensorflow.autolog()` before kicking off the training, then the backend will automatically log the metrics into the server you configured earlier. In our case, Databricks CE.

In [ ]:
mlflow.tensorflow.autolog()

model.fit(x=train_ds, epochs=3)

2023/08/23 22:04:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2b3989f649df4708b0b6af05ee37775b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
  1/469 [..............................] - ETA: 23s - loss: 0.0450 - sparse_categorical_accuracy: 0.9844

469/469 [==============================] - 4s 8ms/step - loss: 0.0403 - sparse_categorical_accuracy: 0.9881
Epoch 2/10
469/469 [==============================] - 4s 9ms/step - loss: 0.0376 - sparse_categorical_accuracy: 0.9878
Epoch 3/10
469/469 [==============================] - 4s 7ms/step - loss: 0.0358 - sparse_categorical_accuracy: 0.9885
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 0.0353 - sparse_categorical_accuracy: 0.9886
Epoch 5/10
469/469 [==============================] - 4s 9ms/step - loss: 0.0335 - sparse_categorical_accuracy: 0.9896
Epoch 6/10
469/469 [==============================] - 4s 8ms/step - loss: 0.0307 - sparse_categorical_accuracy: 0.9904
Epoch 7/10
469/469 [==============================] - 4s 7ms/step - loss: 0.0316 - sparse_categorical_accuracy: 0.9900
Epoch 8/10
469/469 [==============================] - 4s 9ms/step - loss: 0.0288 - sparse_categorical_accuracy: 0.9905
Epoch 9/10
469/469 [==============================] - 5s 11

2023/08/23 22:05:18 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: tuple index out of range
2023/08/23 22:05:18 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.6.0/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2023/08/23 22:05:18 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2023/08/23 22:05:29 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.4.14+cuda11.cudnn86) contains a local version label (+cuda11.cudnn86). MLflow logged a pip requirement for this p

Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

2023/08/23 22:05:29 INFO mlflow.store.artifact.databricks_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

While your training is ongoing, you can find this training in your dashboard. Log in to your [Databricks CE](https://community.cloud.databricks.com/) account, and click on top left to select machine learning in the drop down list. Then click on the experiment icon. See the screenshot below:
![landing page](https://drive.google.com/uc?export=view&id=1QxVaolr-L-w96pKUOiYQut3aSRE-04tC)

After clicking the `Experiment` button, it will bring you to the experiment page, where you can find your runs. Clicking on the most recent experiment and run, you can find your metrics there, similar to:
![experiment page](https://drive.google.com/uc?export=view&id=1wrkSsrGWWQc-ykkGrH86lTzP_66HGhID)


You can click on metrics to see the chart.

Let's evaluate the training result.

In [ ]:
score = model.evaluate(test_ds)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

### Log with MLflow Callback

Auto logging is powerful and convenient, but if you are looking for a more native way as Tensorflow pipelines, you can use `MLflowMetricsLoggingCallback` inside `model.fit()`, it will log:
- Your model configuration, layers, hyperparameters and so on.
- The training stats, including losses and metrics configured with `model.compile()`.

In [ ]:
from mlflow.tensorflow.callbacks import MLflowMetricsLoggingCallback

with mlflow.start_run() as run:
    run_id = run.info.run_id
    model.fit(
        x=train_ds,
        epochs=2,
        callbacks=[MLflowMetricsLoggingCallback(run_id)],
    )

Epoch 1/2
469/469 [==============================] - 18s 14ms/step - loss: 0.3444 - sparse_categorical_accuracy: 0.8945
Epoch 2/2
469/469 [==============================] - 4s 9ms/step - loss: 0.1067 - sparse_categorical_accuracy: 0.9672


Going to the Databricks CE experiment view, you will see a similar dashboard as before.

### Customize the MLflow Callback

If you want to add extra logging logic, you can customize the MLflow callback. You can either subclass from `keras.callbacks.Callback` and write everything from scratchm or subclass from `mlflow.tensorflow.MLflowMetricsLoggingCallback` to change the part you as you need.

Let's look at an example that we want to replace the loss with its log value to log to MLflow.

In [ ]:
import math

# Create our own callback by subclassing `MLflowMetricsLoggingCallback`.
class MLflowCustomCallback(MLflowMetricsLoggingCallback):
    def on_epoch_end(self, epoch, logs=None):
        if not self.log_every_epoch:
            return
        loss = logs["loss"]
        logs["log_loss"] = math.log(loss)
        del logs["loss"]
        self.metrics_logger.record_metrics(logs, epoch)

Train the model with the new callback.

In [ ]:
with mlflow.start_run() as run:
    run_id = run.info.run_id
    model.fit(
        x=train_ds,
        epochs=2,
        callbacks=[MLflowCustomCallback(run_id)],
    )

Epoch 1/2
469/469 [==============================] - 5s 10ms/step - loss: 0.0803 - sparse_categorical_accuracy: 0.9750 - log_loss: -2.5218
Epoch 2/2
469/469 [==============================] - 3s 7ms/step - loss: 0.0681 - sparse_categorical_accuracy: 0.9792 - log_loss: -2.6865


Going to your Databricks CE page, you should find the `log_loss` is replacing the `loss` metric, similar as the screen shot below.

![log loss screenshot](https://drive.google.com/uc?export=view&id=1xp5Q5igViPsx-6Rq5i6_3rv6MXpjdLnm)